<a href="https://colab.research.google.com/github/teoberto/FirstSteps/blob/main/CNAE_Subclasses_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classificação Nacional de Atividades Econômicas (CNAE)**, de gestão do IBGE.

Extração dos dados da planilha [CNAE-Subclasses 2.3](https://concla.ibge.gov.br/images/concla/documentacao/CNAE_Subclasses_2_3_Estrutura_Detalhada.xlsx).

A CNAE é estruturada de forma hierarquizada em cinco níveis, seções,divisões, grupos, classes e subclasses.


---


Exemplo:

Seção: A - Agricultura, pecuária, produção fl orestal, pesca e aqüicultura

Divisão: 01 - Agricultura, pecuária e serviços relacionados

Grupo: 01.1 - Produção de lavouras temporárias

Classe: 01.11-3 - Cultivo de cereais

Subclasse: 0111-3/01 - Cultivo de arroz


In [4]:
import pandas as pd

In [5]:
# Caso seja possível o acesso direto da URL
diretorio = 'https://concla.ibge.gov.br/images/concla/documentacao/CNAE_Subclasses_2_3_Estrutura_Detalhada.xlsx'

In [6]:
# Diretorio local
diretorio = '/diretorio/CNAE_Subclasses_2_3_Estrutura_Detalhada.xlsx'

In [7]:
df = pd.read_excel(diretorio, header=3, dtype=str)

In [8]:
# Nomeia coluna
df.columns = ['cd_secao', 'cd_divisao', 'cd_grupo', 'cd_classe','cd_subclasse', 'ds_denominacao']

In [9]:
# Substitui ponto, traço e barra por vazio
col_remove_ponto = ['cd_grupo', 'cd_classe','cd_subclasse']

for coluna in col_remove_ponto:
    df[coluna] = df[coluna].str.replace('[\./-]', '', regex=True)

In [10]:
# Preencher a Seção para cada subclasse
df['cd_secao'].fillna(method='ffill', inplace=True)

In [11]:
# Data frame específico - DIVISÃO
df_divisao = df.dropna(subset=['cd_divisao'])
df_divisao = df_divisao[['cd_divisao','ds_denominacao']]
df_divisao.rename(columns={'ds_denominacao': 'ds_divisao'}, inplace=True)

In [12]:
# Data frame específico - GRUPO
df_grupo = df.dropna(subset=['cd_grupo'])
df_grupo = df_grupo[['cd_grupo','ds_denominacao']]
df_grupo.rename(columns={'ds_denominacao': 'ds_grupo'}, inplace=True)

In [13]:
# Data frame específico - CLASSE
df_classe = df.dropna(subset=['cd_classe'])
df_classe = df_classe[['cd_classe','ds_denominacao']]
df_classe.rename(columns={'ds_denominacao': 'ds_classe'}, inplace=True)

In [14]:
# Data frame principal - SUBCLASSE
df_subclasse = df.dropna(subset=['cd_subclasse'])
df_subclasse = df_subclasse[['cd_subclasse','ds_denominacao','cd_secao']]
df_subclasse.rename(columns={'ds_denominacao': 'ds_subclasse'}, inplace=True)

# divisao 2, grupo 3, classe 5
df_subclasse['cd_divisao'] = df_subclasse['cd_subclasse'].str[:2]
df_subclasse['cd_grupo'] = df_subclasse['cd_subclasse'].str[:3]
df_subclasse['cd_classe'] = df_subclasse['cd_subclasse'].str[:5]

In [15]:
# Associa divisão, grupo e classe para cada subcnae
df_subclasse = df_subclasse\
  .merge(df_divisao, on='cd_divisao', how = 'left')\
  .merge(df_grupo, on='cd_grupo', how = 'left')\
  .merge(df_classe, on='cd_classe', how = 'left')

In [16]:
#df_subclasse.isnull().sum()
#df_subclasse.nunique()

In [17]:
# Grava arquivo final
df_subclasse.to_csv('/diretorio/tb_cnae_subclasse.csv', index=False, sep=';', encoding='iso-8859-1')